[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/aoi_nasnetlarge.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  TAHU PowerPoint Template.pdf	      Untitled3.ipynb
aoi_test	     TAHU PowerPoint Template - 柱狀圖 1.ods  Untitled4.ipynb
Capstone_PixNet.zip  test for excel			      Untitled5.ipynb
Colab Notebooks      Untitled0.ipynb			      Untitled6.ipynb
PiXNet		     Untitled1.ipynb			      Untitled7.ipynb
Sample upload.txt    Untitled2.ipynb


In [0]:
!cp drive/aoi_test/train_images.zip .

In [0]:
!cp drive/aoi_test/train.csv .

In [7]:
!unzip train_images.zip


Archive:  train_images.zip
   creating: train_images/
 extracting: train_images/train_00000.png  
 extracting: train_images/train_00001.png  
 extracting: train_images/train_00002.png  
 extracting: train_images/train_00003.png  
 extracting: train_images/train_00004.png  
 extracting: train_images/train_00005.png  
 extracting: train_images/train_00006.png  
 extracting: train_images/train_00007.png  
 extracting: train_images/train_00008.png  
 extracting: train_images/train_00009.png  
 extracting: train_images/train_00010.png  
 extracting: train_images/train_00011.png  
 extracting: train_images/train_00012.png  
 extracting: train_images/train_00013.png  
 extracting: train_images/train_00014.png  
 extracting: train_images/train_00015.png  
 extracting: train_images/train_00016.png  
 extracting: train_images/train_00017.png  
 extracting: train_images/train_00018.png  
 extracting: train_images/train_00019.png  
 extracting: train_images/train_00020.png  
 extracting: train_ima

In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image

from keras.layers import Dense, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout,Cropping2D,Activation
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import Callback
from keras.applications.mobilenet import MobileNet
from keras.applications.nasnet import NASNetMobile,NASNetLarge


Using TensorFlow backend.


In [0]:
src = 'train_images/'
src2=''

In [4]:
train_img = os.listdir('train_images')
print ('iamge lies size', len(train_img))
img_list=[]
for i,img in enumerate(train_img):
    x = cv2.imread(src+img)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    #x = x/127.5 -1.
    img_list.append(x)
    
print ('done')

iamge lies size 2528
done


In [5]:
train_data = np.array(img_list)
print (train_data.shape)
#train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 512, 512, 3)
(2528, 512, 512, 3)


In [6]:
print ('here')

here


In [7]:
df_y = pd.read_csv('train.csv')
print (df_y['Label'].value_counts())
train_y =  df_y['Label'].values

onehot_encoder = OneHotEncoder(sparse=False)
train_y = train_y.reshape(len(train_y), 1)
print (train_y.shape)
train_label = onehot_encoder.fit_transform(train_y)


print ('beofre', train_label.shape)
#print (df_y['Label'][:10])
#print (train_label[:10,:])

train_label = train_label[:,np.newaxis,np.newaxis,:]

print ('after', train_label.shape)


0    674
5    644
1    492
3    378
4    240
2    100
Name: Label, dtype: int64
(2528, 1)
beofre (2528, 6)
after (2528, 1, 1, 6)


In [0]:
class CollectOutputAndTarget(Callback):
    def __init__(self):
        super(CollectOutputAndTarget, self).__init__()
        self.targets = []  # collect y_true batches
        self.outputs = []  # collect y_pred batches

        # the shape of these 2 variables will change according to batch shape
        # to handle the "last batch", specify `validate_shape=False`
        self.var_y_true = tf.Variable(0., validate_shape=False)
        self.var_y_pred = tf.Variable(0., validate_shape=False)

    def on_epoch_end(self, batch, logs=None):
        # evaluate the variables and save them into lists
        '''
        print ('\n 0', len(self.validation_data))
        print ('1',  self.validation_data[0].shape )
        print ('2',  self.validation_data[1].shape )
        print ( '3', self.validation_data[2].shape )
        
        print ('4', test.shape)
       
        print ('5', test2.shape)
        
        print ('6', test3.shape)
        '''
        test = self.model.predict(self.validation_data[0])
        test2 = np.squeeze(test)
        test3 = np.argmax(test2,axis=1)
        print ('y_pred = \n', test3[:30])
        test_4 = np.squeeze(self.validation_data[1])
        test_5 = np.argmax(test_4,axis=1)
        print ('y_true = \n', test_5[:30])
        
        #print ('6', np.argmax(np.squeeze(test),axis=1))
        #out_result = np.argmax(np.squeeze(self.model.predict(self.validation_data[0])),axis=1)
        #ground_truth = np.argmax(np.squeeze (self.validation_data[1]), axis=1)



In [9]:


inputs = Input(shape=(512,512,3))
inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
pool0 = x = MaxPooling2D(pool_size=(2,2))(inputs)
print ('ppol0', pool0.get_shape())
inputs_mobnet = Cropping2D(cropping=16,data_format = "channels_last")(pool0)
print ('inputs_mobnet', inputs_mobnet.get_shape())
# create the base pre-trained model
base_model = NASNetLarge(weights='imagenet',input_tensor=inputs_mobnet,input_shape=(224,224,3), include_top=False)
for layer in base_model.layers:
    layer.trainable = False

ppol0 (?, 256, 256, 3)
inputs_mobnet (?, 224, 224, 3)


In [10]:
# add a global spatial average pooling layer
x = base_model.output
print ('shape of mobilenet output', x.get_shape())
#x = MaxPooling2D(pool_size=(2,2))(x)

#x = GlobalAveragePooling2D()(x)
#print ('shape of pooling output', x.get_shape())
# let's add a fully-connected layer
x = Conv2D(32,(3,3), activation='relu', padding='valid')(x)
x = Conv2D(128,(3,3), activation='relu', padding='valid')(x)
x = Conv2D(128,(3,3), activation='relu', padding='valid')(x)
print ('shape of extra conv output', x.get_shape())
#x = Conv2D(128,(2,2), activation='relu', padding='valid')(x)

#predictions = Conv2D(6,(1,1), activation='relu', padding='same')(x)
x = Dense(256, activation='relu')(x)

# add Dropout
x = Dropout(0.3)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)
#predictions = Reshape(target_shape=(1,1,6))(predictions)
print ('final output', predictions.get_shape())

shape of mobilenet output (?, 7, 7, 4032)
shape of extra conv output (?, 1, 1, 128)
final output (?, 1, 1, 6)


In [11]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 256, 256, 3)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
cropping2d_1 (Cropping2D)       (None, 224, 224, 3)  0           max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
stem_conv1

In [0]:

def my_metrics(y_true,y_pred):
    return y_pred


In [0]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

#output_layers = ['predictions']
#model.metrics_tensors += output_layers


In [20]:

# initialize the variables and the `tf.assign` ops
cbk = CollectOutputAndTarget()
'''
fetches = [tf.assign(cbk.var_y_true, model.targets[0], validate_shape=False),
           tf.assign(cbk.var_y_pred, model.outputs[0], validate_shape=False)]

model._function_kwargs = {'fetches': fetches}  # use `model._function_kwargs` if using `Model` instead of `Sequential`
'''

"\nfetches = [tf.assign(cbk.var_y_true, model.targets[0], validate_shape=False),\n           tf.assign(cbk.var_y_pred, model.outputs[0], validate_shape=False)]\n\nmodel._function_kwargs = {'fetches': fetches}  # use `model._function_kwargs` if using `Model` instead of `Sequential`\n"

In [0]:
src= 'drive/aoi_test/'

In [0]:
# train the model on the new data for a few epochs
use_model = False

if use_model:
    
    if os.path.isfile(src+'nasnet_large_1.h5'):

      model = load_model(src+'nasnet_large_1.h5')
      print ('model laoded')
    else:
      print ('model not exist')


In [24]:
checkpoint = ModelCheckpoint(src+'nasnet_large_1.h5',monitor = 'val_loss', save_best_only=True, mode= 'auto')
#callback_list = [cbk, checkpoint]
earlystop = EarlyStopping(patience=20)
callback_list = [checkpoint, earlystop]
#callback_list = [checkpoint]
for i in range (20):


  
  train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)
  
  model.fit(x=train_x, y=train_y, batch_size=32, epochs=100, verbose=1, callbacks=callback_list, validation_split=0.2, shuffle=True)
  #print ('epoch', i, 'shape=', len(cbk.outputs))


  model.save(src+'nasnet_large_30_1.h5')
  print ('epoch', i, 'model saved')







Train on 1617 samples, validate on 405 samples
Epoch 1/100
1617/1617 [==============================] - 67s 41ms/step - loss: 1.6869 - acc: 0.2585 - val_loss: 1.6570 - val_acc: 0.2469
Epoch 2/100
1617/1617 [==============================] - 67s 42ms/step - loss: 1.6795 - acc: 0.2566 - val_loss: 1.6543 - val_acc: 0.2444
Epoch 3/100
1617/1617 [==============================] - 67s 42ms/step - loss: 1.6589 - acc: 0.2789 - val_loss: 1.6626 - val_acc: 0.2469
Epoch 4/100
1617/1617 [==============================] - 67s 41ms/step - loss: 1.6493 - acc: 0.2721 - val_loss: 1.6487 - val_acc: 0.2469
Epoch 5/100
1617/1617 [==============================] - 67s 42ms/step - loss: 1.6449 - acc: 0.2900 - val_loss: 1.6637 - val_acc: 0.2519
Epoch 6/100
1617/1617 [==============================] - 67s 41ms/step - loss: 1.6244 - acc: 0.3135 - val_loss: 1.6573 - val_acc: 0.2568
Epoch 7/100
1617/1617 [==============================] - 67s 41ms/step - loss: 1.5861 - acc: 0.3253 - val_loss: 1.6493 - val_acc: 0

KeyboardInterrupt: ignored

In [26]:
 model.save('nasnet_large_30_1.h5')
 print ('epoch', i, 'model saved')
from google.colab import files
files.download('nasnet_large_30_1.h5') 

epoch 3 model saved


----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 60598, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [12]:
nas_check = ['nasnet_large_1.h5','nasnet_large_30_1.h5']


for m in nas_check:


  if os.path.isfile(m):

    model = load_model(m)
    print ('model laoded')
  else:
    print ('model not exist')
  
 
  train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)
  
  re=model.evaluate(val_x,val_y)
  print ('model=',m, 'result=', re)


model laoded
506/506 [==============================] - 30s 58ms/step
model= nasnet_large_1.h5 result= [1.824063992783015, 0.20750988148182278]
model laoded
506/506 [==============================] - 31s 60ms/step
model= nasnet_large_30_1.h5 result= [1.821973622080837, 0.20355731284194312]


In [0]:
def shuffle_list(s,l):
    final_list = []
    n_list = np.arange(l)
    np.random.shuffle(n_list)
    b_s = int(np.floor(l/s))
    #print (b_s)
    
    for i in range (b_s):
        #print (i)
        final_list.append(n_list[i*s:(i+1)*s])
    return final_list

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)

l = len(train_x)
s= 32

for epoch in range (100):

  batch_list = shuffle_list(s,l)
  for batch in batch_list:
    
    X = train_x[batch]
    Y = train_y[batch]
    [loss, accuracy]= model.train_on_batch(X,Y)

    print('during training: loss is {} accuracy is {}'.format(loss, accuracy))
    print ('prediction')
    print (np.argmax(np.squeeze(model.predict(X)),axis=-1))
    print ('ground truth')
    print (np.argmax(np.squeeze(Y),axis=-1))
  model.save(src+'densenet_30_test_2.h5')
  print ('epoch', epoch, 'model saved')
